In [ ]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

    8192/17225924 [..............................] - ETA: 7:59:11

In [ ]:
def addTopModelMobileNet(bottom_layer,num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

# Let's add our FC Head back onto MobileNet

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Cpnv2D, MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/monkey_breed/train'
validation_data_dir = '/monkey_breed/validation'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valdation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_direcotry(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical'
)

validation_generator = valdation_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical'
)

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,EarlyStopping

chackpoint = ModelCheckpoint('monkey_breed_mobileNet.h5',
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1
                            )

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose =1,
                          restore_best_weights = True
                         )
callbacks = [earlystop,checkpoint]

model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics = ['accuracy']
             )


In [ ]:
# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epochs = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples//batch_size
)

In [ ]:
from keras.models import load_model

classifier = load_model('monkey_breed_mobileNet.h5')

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os import isfile, join

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

In [ ]:
def draw_test(name, pred,im)
monkey = monkey_breed_dict[str(pred)]
BLACK = [0,0,0]
expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK))
cv2.putText(expanded_image,monkey,(20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)  #(20, 60)_coordinate, 1_text_type, 2_thickness
cv2.imshow(name,expanded_image)

In [ ]:
def getRandomImage(path):
     """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name) 
    

In [ ]:
for i in range(0,10):
    input_im =getRandomImage("/monkey_breed/validation/")
    input_orginal = input_im.copy()
    input_orginal = cv2.resize(input_original,None,fx=0.5,fy=0.5,interpolation = cv2.INTER_LINEAR )
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im/255.
    input_im = input_im.reshape(1,244,244,3)
    
    res = np.argmax(classifier.predict(input_im,1,verbose = 0),axsi =1)
    
    draw_test("Prediction",res,input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()